<a href="https://colab.research.google.com/github/RaiAnant/MangaChroma/blob/master/fasterai/generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/RaiAnant/MangaChroma.git
!pip install import_ipynb
import import_ipynb

Cloning into 'MangaChroma'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 111 (delta 44), reused 3 (delta 0), pack-reused 0
Receiving objects: 100% (111/111), 25.68 KiB | 410.00 KiB/s, done.
Resolving deltas: 100% (44/44), done.
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=a83259b81f3335de56534f129af531fe0bf397aca74b9a5c0f8433672a847246
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


## Imports

In [2]:
from fastai.vision import *
from fastai.vision.learner import cnn_config
from MangaChroma.fasterai.unet import DynamicUnetWide, DynamicUnetDeep
from MangaChroma.fasterai.loss import FeatureLoss
# from MangaChroma.fasterai.dataset import *

importing Jupyter notebook from /content/MangaChroma/fasterai/unet.ipynb
fatal: destination path 'MangaChroma' already exists and is not an empty directory.
importing Jupyter notebook from /content/MangaChroma/fasterai/layers.ipynb
importing Jupyter notebook from /content/MangaChroma/fasterai/loss.ipynb
Sun Aug 18 15:04:38 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |     11MiB / 11441MiB |      0%      Default |
+----------------------

In [0]:
# For getting the model prepared at inference time
def gen_inference_wide(root_folder:Path, wights_name:str, nf_factor:int=2, arch=models.resnet101)->Learner:
  data = get_dummy_databunch()
  learn = gen_learner_wide(data=data, gen_loss=F1.l1_loss, nf_factor=nf_factor, arch=arch)
  learn.path = root_folder
  learn.load(weights_name)
  learn.model.eval()
  return learn

In [0]:
def unet_learner_wide(data:DataBunch, arch:Callable, pretrained:bool=True, blur_final:bool=True,
                      norm_type:Optional[NormType]=NormType, split_on:Optional[SplitFuncOrIdxList]=None,
                      blur:bool=False, self_attention:bool=False, y_range:Optional[Tuple[float,float]]=None, last_cross:bool=True,
                      bottle:bool=False, nf_factor:int=1, **kwargs:Any)->Learner:
  """Build Unet learner from `data` and `arch`."""
  #Get the metadata associated with `arch`.
  meta = cnn_config(arch)
  body = create_body(arch, pretrained)    
  model = to_device(DynamicUnetWide(body, n_classes=data.c, blur=blur, blur_final=blur_final,
                    self_attention=self_attention, y_range=y_range, norm_type=norm_type, last_cross=last_cross,
                    bottle=bottle, nf_factor=nf_factor), data.device)
  learn = Learner(data, model, **kwargs)
  learn.split(ifnone(split_on,meta['split']))
  if pretrained: learn.freeze()
  apply_init(model[2], nn.init.kaiming_normal_)
  return learn

In [0]:
def get_learner_wide(data:ImageDataBunch, gen_loss=FeatureLoss(), arch=models.resnet101, nf_factor:int=2)->Learner:
  return unet_learner_wide(data, arch=arch, wd=1e-3, blur=True, norm_type=NormType.Spectral, 
                           self_attention=True, y_range=(-3.,3.), loss_func=gen_loss, nf_factor=nf_factor)

In [0]:
def gen_inference_deep(root_folder:Path, weights_name:str, arch=models.resnet34, nf_factor:float=1.5)->Learner:
  data = get_dummy_databunch()
  learn = gen_learner_deep(data=data, gen_loss=F.l1_loss, arch=arch, nf_factor=nf_factor)
  learn.path = root_folder
  learn.load(weights_name)
  learn.model.eval()
  return learn

In [0]:
def unet_learner_deep(data:DataBunch, arch:Callable, pretrained:bool=True, blur_final:bool=True,
                 norm_type:Optional[NormType]=NormType, split_on:Optional[SplitFuncOrIdxList]=None, 
                 blur:bool=False, self_attention:bool=False, y_range:Optional[Tuple[float,float]]=None, last_cross:bool=True,
                 bottle:bool=False, nf_factor:float=1.5, **kwargs:Any)->Learner:
  "Build Unet learner from `data` and `arch`."
  meta = cnn_config(arch)
  body = create_body(arch, pretrained)
  model = to_device(DynamicUnetDeep(body, n_classes=data.c, blur=blur, blur_final=blur_final,
        self_attention=self_attention, y_range=y_range, norm_type=norm_type, last_cross=last_cross,
        bottle=bottle, nf_factor=nf_factor), data.device)
  learn = Learner(data, model, **kwargs)
  learn.split(ifnone(split_on,meta['split']))
  if pretrained: learn.freeze()
  apply_init(model[2], nn.init.kaiming_normal_)
  return learn

In [0]:
def gen_learner_deep(data:ImageDataBunch, gen_loss=FeatureLoss(), arch=models.resnet34, nf_factor:float=1.5)->Learner:
  return unet_learner_deep(data, arch, wd=1e-3, blur=True, norm_type=NormType.Spectral,
                      self_attention=True, y_range=(-3.,3.), loss_func=gen_loss, nf_factor=nf_factor)